<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <span style="color: #641E16">Contexte</span>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été captées via l'outil d’export de données ***stackexchange explorer***, qui recense un grand nombre de données authentiques de la plateforme d’entraide.     
Elles portent sur la période 2009 / 2020 et **uniquement sur les posts "de qualité"** ayant au minimum 1 réponse, 5 commentaires, 20 vues et un score supérieur à 5.

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **data cleaning et exploration des données**. Un second notebook traitera ensuite les approches supervisées et non supervisées pour traiter la création de Tags à partir des données textuelles.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <span style="color:#641E16">Sommaire</span>
1. [Importation et description des données](#section_1)
2. [Data exploration](#section_2)

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Import Python libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

In [ ]:
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")

## <span style="color: #641E16" id="section_1">Importation et description des données</span>

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'

# Concat all CSV datasets in one Pandas DataFrame
df_columns = pd.read_csv(path+'StackOverflow_questions_2009.csv').columns
data = pd.DataFrame(columns=df_columns)
for f in os.listdir(path):
    temp = pd.read_csv(path+f)
    data = pd.concat([data, temp], 
                     axis=0,
                     ignore_index=True)
data.head(3)

In [ ]:
# Print full dataset infos
data.info()

In [ ]:
# Describe data
data.describe()

Le jeu de données ne compte pas de valeurs nulles. La variable Id ne compte que des valeurs uniques, nous pouvons donc l'utiliser en index :

In [ ]:
data.set_index('Id', inplace=True)

## <span style="color: #641E16" id="section_2">Data exploration</span>

Dans un premier temps, nous allons regarder l'**évolution du nombre de questions par année** dans notre jeu de données.

In [ ]:
# Convert CreationDate to datetime format
data['CreationDate'] = pd.to_datetime(data['CreationDate'])

# Grouper with 1 year delta
post_year = data.groupby(pd.Grouper(key='CreationDate',
                                    freq='1Y')).agg({'Title': 'count'})

# Plot evolution
fig = plt.figure(figsize=(15,6))
sns.lineplot(data=post_year, x=post_year.index, y='Title')
plt.axhline(post_year.Title.mean(), 
            color="r", linestyle='--',
            label="Mean of question per year : {:04d}"\
                   .format(int(post_year.Title.mean())))
plt.xlabel("Date of questions")
plt.ylabel("Number of questions")
plt.title("Number of questions evolution from 2009 to 2020",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

On remarque ici que sur nos critères de sélection, le nombre de questions posées a tendance à diminuer de manière constante depuis 2014.

Nous allons à présent vérifier la **longeur des différents titres** de la base :

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=data.Title.str.len())
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 5))
plt.axvline(data.Title.str.len().median() - data.Title.str.len().min(),
            color="r", linestyle='--',
            label="Title Lenght median : "+str(data.Title.str.len().median()))
ax.set_xlabel("Lenght of title")
plt.title("Title lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

Nous allons également ploter la répartition des **longueurs de la variable** `Body` *(les corps de texte des questions)*. L'étendue étant très importante, nous allons dans un premier temps **discrétiser ces longueur** pour ne pas surcharger les temps de calculs de projection graphique :

In [ ]:
# Discretizer for Body characters lenght
X = pd.DataFrame(data.Body.str.len())

# Sklearn discretizer with 200 bins
discretizer = KBinsDiscretizer(n_bins=200,
                               encode='ordinal',
                               strategy='uniform')
body_lenght = discretizer.fit_transform(X)
body_lenght = discretizer.inverse_transform(body_lenght)
body_lenght = pd.Series(body_lenght.reshape(-1))

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=body_lenght)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 25))
ax.set_xlabel("Lenght of Body (after discretization)")
plt.title("Body lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

On remarque que la majeur partie des questions compte moins de 4000 caractères *(balises HTML compris)* mais certains posts dépassent les 31 000 caractères. Nous allons **filtrer notre jeu de données pour conserver uniquement les questions de moins de 10 000 caractères** afin de ne pas compliquer le NLP plus que nécessaire.

In [ ]:
# Filter data on body lenght
data = data[data.Body.str.len() < 10000]
data.shape

### Analyse des tags
Nous allons faire une rapide analyse exploratoire sur les tags du jeu de données.

In [ ]:
data['Tags'].head(3)

Nous allons modifier les séparateurs de Tags pour favoriser les extractions :

In [ ]:
# Replace open and close balise between tags
data['Tags'] = data['Tags'].str.translate(str.maketrans({'<': '', '>': ','}))

# Delete last "," for each row
data['Tags'] = data['Tags'].str[:-1]
data['Tags'].head(3)

Les tags contenus dans la variable `Tags` sont ensuite splités et ajoutés dans une liste pour ensuite les classer :

In [ ]:
def count_split_tags(df, column, separator):
    """This function allows you to split the different words contained
    in a Pandas Series cell and to inject them separately into a list.
    This makes it possible, for example, to count the occurrences of words.

    Parameters
    ----------------------------------------
    df : Pandas Dataframe
        Dataframe to use.
    column : string
        Column of the dataframe to use
    separator : string
        Separator character for str.split.
    ----------------------------------------
    """
    list_words = []
    for word in df[column].str.split(separator):
        list_words.extend(word)
    df_list_words = pd.DataFrame(list_words, columns=["Tag"])
    df_list_words = df_list_words.groupby("Tag")\
        .agg(tag_count=pd.NamedAgg(column="Tag", aggfunc="count"))
    df_list_words.sort_values("tag_count", ascending=False, inplace=True)
    return df_list_words

In [ ]:
tags_list = count_split_tags(df=data, column='Tags', separator=',')
print("Le jeu de données compte {} tags.".format(tags_list.shape[0]))

In [ ]:
# Plot the results of splits
fig = plt.figure(figsize=(15, 8))
sns.barplot(data=tags_list.iloc[0:40, :],
            x=tags_list.iloc[0:40, :].index,
            y="tag_count", color="#f48023")
plt.xticks(rotation=90)
plt.title("40 most popular tags in Stackoverflow (2009 - 2020)",
          fontsize=18, color="#641E16")
plt.show()

Dans les 40 tags les plus populaires sur StackOverflow, les tags **C++**, **C#** et **java** sont sans surprise dans le top 3. Le dataset compte **plus de 16 800 tags** différents pour la période 2009 - 2020. 

Il peut être intéressant de regarder si ces tags populaires ont évolués au fil du temps. Prenons par exemple les années 2009, 2012, 2016 et 2020 pour vérifier.

In [ ]:
# Subplots parameters
years = {0: 2009, 1: 2012, 2: 2016, 3: 2020}
colors = {0: "#f48023", 1: "#d16e1e",
          2: "#b25d19", 3: "#904b14"}
subplots = 4
cols = 2
rows = subplots // cols
rows += subplots % cols
position = range(1, subplots + 1)

# Plot popular tags for each year
fig = plt.figure(1, figsize=(20, 16))
for k in range(subplots):
    subset = data[data["CreationDate"].dt.year == years[k]]
    temp_list = count_split_tags(df=subset, column='Tags', separator=',')
    ax = fig.add_subplot(rows, cols, position[k])
    sns.barplot(data=temp_list.iloc[0:20, :],
            x=temp_list.iloc[0:20, :].index,
            y="tag_count", color=colors[k])
    plt.xticks(rotation=90)
    ax.set_title("20 most popular tags for {}".format(years[k]),
                 fontsize=18, color="#641E16")

fig.tight_layout()
plt.show()

On remarque en effet que les centres d'intérêt évoluent en fonction des années. Cependant, on retrouve les principaux languages et framework informatiques dans les premières places.

In [ ]:
pep8(_ih)